# Noise analysis with Cirq + devqubit

This notebook is a **small, practical demo** of Cirq's noisy simulation tools and devqubit's experiment tracking.

We will:

1. Build a simple **GHZ** circuit (highly sensitive to noise).
2. Run an **ideal baseline** and then a **noise-model sweep**.
3. Study **fidelity vs. circuit depth** to show noise accumulation.
4. Use devqubit's comparison utilities to compute a simple **distribution distance** (TVD).

> Goal: keep the code minimal while illustrating the essential pieces:
> - **Noise channels** (e.g., depolarizing / amplitude damping / phase damping / bit-flip)
> - **Noise models** (inserting a channel after each operation)
> - **Mixed-state simulation** via `DensityMatrixSimulator`


In [ ]:
from __future__ import annotations

from importlib.metadata import entry_points


def has_adapter(name: str) -> bool:
    eps = entry_points().select(group="devqubit.adapters")
    return any(ep.name == name for ep in eps)


if not has_adapter("cirq"):
    raise ImportError(
        "devqubit Cirq adapter is not installed.\n"
        "Install with: pip install 'devqubit[cirq]'"
    )

print("Cirq adapter available!")

In [ ]:
from dataclasses import dataclass
from typing import Dict, Optional, Tuple
from pathlib import Path
import shutil

import cirq
import numpy as np

from devqubit import (
    track,
    wrap_backend,
    create_registry,
    create_store,
    diff,
)

In [ ]:
"""Setup: config, workspace, and devqubit store/registry."""

@dataclass(frozen=True)
class ExperimentConfig:
    """Configuration for the noise analysis demo.

    Parameters
    ----------
    project_name:
        devqubit project name used for run grouping.
    workspace_dir:
        Local directory for the devqubit file-based registry and object store.
    seed:
        Global seed for reproducibility of any NumPy-driven randomness.

    n_qubits_baseline:
        Number of qubits for the baseline GHZ run.
    shots_baseline:
        Number of samples (repetitions) for baseline measurement statistics.

    noise_models:
        Iterable of `(noise_type, error_rate)` tuples used in the sweep.
        `noise_type="ideal"` means no noise.
    n_qubits_depth:
        Number of qubits for the depth-scaling experiment.
    shots_depth:
        Samples per depth point.
    depth_repetitions:
        Entangling-layer repetition counts (higher = deeper circuit).
    depth_noise_type:
        Noise channel used in the depth-scaling experiment.
    depth_error_rate:
        Error rate for the depth-scaling experiment.
    """

    project_name: str = "cirq_noise_analysis"
    workspace_dir: str = ".devqubit_cirq_noise_demo"
    seed: int = 42

    n_qubits_baseline: int = 5
    shots_baseline: int = 10_000

    noise_models: Tuple[Tuple[str, float], ...] = (
        ("ideal", 0.0),
        ("depolarizing", 0.01),
        ("amplitude_damping", 0.02),
        ("bit_flip", 0.01),
    )

    n_qubits_depth: int = 4
    shots_depth: int = 5_000
    depth_repetitions: Tuple[int, ...] = (1, 2, 3, 5, 8)
    depth_noise_type: str = "depolarizing"
    depth_error_rate: float = 0.005


CFG = ExperimentConfig()

# Reproducibility for any NumPy randomness used in analysis
np.random.seed(CFG.seed)

# ----------------------------
# devqubit workspace setup
# ----------------------------

WORKSPACE = Path(CFG.workspace_dir)
if WORKSPACE.exists():
    shutil.rmtree(WORKSPACE)
WORKSPACE.mkdir(parents=True, exist_ok=True)

store = create_store(f"file://{WORKSPACE}/objects")
registry = create_registry(f"file://{WORKSPACE}")

print(f"Workspace: {WORKSPACE.resolve()}")
print(f"Project  : {CFG.project_name}")

In [ ]:
"""
Utilities: circuits, simulators, and simple metrics.

Cirq represents noise as quantum channels (Kraus operators under the hood).
For this demo we use built-in single-qubit channels and apply them *after every operation*
via `cirq.ConstantQubitNoiseModel`.
"""


def create_ghz_circuit(n_qubits: int, *, measure_key: str = "m") -> cirq.Circuit:
    """Create an n-qubit GHZ circuit with a final measurement."""

    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()

    circuit.append(cirq.H(qubits[0]))
    for i in range(n_qubits - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))

    circuit.append(cirq.measure(*qubits, key=measure_key))
    return circuit


def create_repeated_ghz_circuit(
    n_qubits: int,
    repetitions: int,
    *,
    measure_key: str = "m",
) -> cirq.Circuit:
    """Create a deeper GHZ-like circuit by repeating entangling layers."""

    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubits[0]))

    for _ in range(repetitions):
        # Forward chain
        for i in range(n_qubits - 1):
            circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
        # Backward chain (adds depth; doesn't aim to "undo" perfectly under noise)
        for i in range(n_qubits - 2, -1, -1):
            circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))

    # Final GHZ entangling pass
    for i in range(n_qubits - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))

    circuit.append(cirq.measure(*qubits, key=measure_key))
    return circuit


def build_simulator(noise_type: str, error_rate: float) -> cirq.SimulatesSamples:
    """Construct a Cirq simulator for the given noise model."""

    if noise_type == "ideal":
        # Pure-state simulation (fast baseline)
        return cirq.Simulator()

    channel_gate_map = {
        "depolarizing": cirq.depolarize(error_rate),
        "amplitude_damping": cirq.amplitude_damp(error_rate),
        "bit_flip": cirq.bit_flip(error_rate),
    }
    if noise_type not in channel_gate_map:
        raise ValueError(f"Unknown noise_type={noise_type!r}")

    noise_model = cirq.ConstantQubitNoiseModel(channel_gate_map[noise_type])
    return cirq.DensityMatrixSimulator(noise=noise_model)


def sample_counts(
    simulator: cirq.SimulatesSamples,
    circuit: cirq.Circuit,
    shots: int,
    *,
    measure_key: str = "m",
) -> Dict[int, int]:
    """Sample bitstring counts from a circuit."""

    result = simulator.run(circuit, repetitions=shots)
    return dict(result.histogram(key=measure_key))


def compute_ghz_fidelity_from_counts(
    counts: Dict[int, int],
    n_qubits: int,
    *,
    shots: Optional[int] = None,
) -> float:
    """Estimate GHZ "fidelity" from measurement counts.

    We use a **very simple** proxy for GHZ quality:
    probability mass on the two ideal outcomes |00...0⟩ and |11...1⟩.
    """
    total = int(shots) if shots is not None else int(sum(counts.values()))
    if total <= 0:
        return 0.0

    all_zeros = 0
    all_ones = (1 << n_qubits) - 1
    good = counts.get(all_zeros, 0) + counts.get(all_ones, 0)
    return good / total


def counts_to_probabilities(counts: Dict[int, int]) -> Dict[int, float]:
    """Convert integer counts to probabilities."""

    total = sum(counts.values())
    if total == 0:
        return {}
    return {k: v / total for k, v in counts.items()}


def total_variation_distance(p: Dict[int, float], q: Dict[int, float]) -> float:
    """Compute Total Variation Distance (TVD) between two discrete distributions."""

    keys = set(p) | set(q)
    return 0.5 * sum(abs(p.get(k, 0.0) - q.get(k, 0.0)) for k in keys)

## 1) GHZ circuit (the test workload)

We use GHZ because it's easy to reason about **ideal outcomes** and it degrades quickly under noise.


In [ ]:
# Build and display the baseline GHZ circuit
ghz_circuit = create_ghz_circuit(CFG.n_qubits_baseline)
print(ghz_circuit)
print(f"Depth (moments): {len(ghz_circuit)}")

## 2) Ideal baseline run (tracked)

We first sample the circuit **without noise**.  
This run becomes the baseline for later comparisons.


In [ ]:
# --- Ideal baseline (no noise) ---
baseline_sim = build_simulator("ideal", 0.0)

with track(
    project=CFG.project_name,
    store=store,
    registry=registry,
) as run:
    # Wrap the simulator backend so devqubit can collect backend-level metadata
    sampler = wrap_backend(run, baseline_sim)

    run.log_params(
        {
            "n_qubits": CFG.n_qubits_baseline,
            "shots": CFG.shots_baseline,
            "noise_type": "ideal",
            "error_rate": 0.0,
            "circuit_depth_moments": len(ghz_circuit),
        }
    )
    run.set_tags({"experiment": "ghz_fidelity", "noise": "ideal", "role": "baseline"})

    counts = sample_counts(sampler, ghz_circuit, CFG.shots_baseline)
    fidelity = compute_ghz_fidelity_from_counts(
        counts,
        CFG.n_qubits_baseline,
        shots=CFG.shots_baseline,
    )

    run.log_metrics(
        {
            "fidelity_proxy": float(fidelity),
            "prob_all_zeros": counts.get(0, 0) / CFG.shots_baseline,
            "prob_all_ones": counts.get((1 << CFG.n_qubits_baseline) - 1, 0)
            / CFG.shots_baseline,
        }
    )
    run.log_json(
        name="counts",
        obj={"counts": counts, "shots": CFG.shots_baseline},
        role="artifact",
    )

    ideal_run_id = run.run_id

print(f"Ideal baseline run_id: {ideal_run_id}")
print(f"Fidelity proxy       : {fidelity:.4f}")

## 3) Noise model sweep

We run the same circuit under several single-qubit noise channels by inserting the channel after each operation.
Each run is tracked and grouped under a single sweep ID.


In [ ]:
def run_ghz_fidelity_experiment(
    *,
    noise_type: str,
    error_rate: float,
    n_qubits: int,
    shots: int,
    group_id: Optional[str] = None,
    group_name: Optional[str] = None,
) -> Tuple[str, float]:
    """Run a GHZ sampling experiment under a specified noise model."""

    circuit = create_ghz_circuit(n_qubits)
    simulator = build_simulator(noise_type, error_rate)

    with track(
        project=CFG.project_name,
        store=store,
        registry=registry,
        group_id=group_id,
        group_name=group_name,
    ) as run:
        sampler = wrap_backend(run, simulator)

        run.log_params(
            {
                "n_qubits": n_qubits,
                "shots": shots,
                "noise_type": noise_type,
                "error_rate": float(error_rate),
                "circuit_depth_moments": len(circuit),
            }
        )
        run.set_tags({"experiment": "ghz_fidelity", "noise": noise_type})

        counts = sample_counts(sampler, circuit, shots)
        fidelity = compute_ghz_fidelity_from_counts(
            counts,
            n_qubits,
            shots=shots,
        )

        run.log_metrics(
            {
                "fidelity_proxy": float(fidelity),
                "prob_all_zeros": counts.get(0, 0) / CFG.shots_baseline,
                "prob_all_ones": counts.get((1 << CFG.n_qubits_baseline) - 1, 0)
                / CFG.shots_baseline,
            }
        )
        run.log_json(
            name="counts",
            obj={"counts": counts, "shots": shots},
            role="artifact",
        )

        return run.run_id, float(fidelity)

In [ ]:
# Run the noise sweep
sweep_group_id = "noise_sweep_example"
sweep_group_name = "Noise Model Comparison"

sweep_results: list[dict] = []

print(f"Sweep group_id: {sweep_group_id}")
print("-" * 60)

for noise_type, error_rate in CFG.noise_models:
    run_id, fid = run_ghz_fidelity_experiment(
        noise_type=noise_type,
        error_rate=error_rate,
        n_qubits=CFG.n_qubits_baseline,
        shots=CFG.shots_baseline,
        group_id=sweep_group_id,
        group_name=sweep_group_name,
    )
    sweep_results.append(
        {
            "noise_type": noise_type,
            "error_rate": float(error_rate),
            "fidelity_proxy": float(fid),
            "run_id": run_id,
        }
    )
    print(f"{noise_type:20s}  rate={error_rate:>7.4f}  fidelity={fid:.4f}")

# Sort best-to-worst for quick reading
sweep_results_sorted = sorted(
    sweep_results, key=lambda r: r["fidelity_proxy"], reverse=True
)

print("\nFidelity ranking (best -> worst):")
for rec in sweep_results_sorted:
    print(f"  {rec['noise_type']:20s}: {rec['fidelity_proxy']:.4f}")

## 4) Fidelity proxy vs. circuit depth

To demonstrate **noise accumulation**, we increase circuit depth by repeating entangling layers and track fidelity decay.


In [ ]:
depth_group_id = "depth_sweep_example"
depth_group_name = "Fidelity vs Depth"

depth_noise_type = CFG.depth_noise_type
depth_error_rate = CFG.depth_error_rate

sim = build_simulator(depth_noise_type, depth_error_rate)

depth_points: list[dict] = []

print(f"Depth sweep group_id: {depth_group_id}")
print(f"Noise: {depth_noise_type} (rate={depth_error_rate})")
print("-" * 60)

with track(
    project=CFG.project_name,
    store=store,
    registry=registry,
    group_id=depth_group_id,
    group_name=depth_group_name,
) as run:
    sampler = wrap_backend(run, sim)

    run.log_params(
        {
            "n_qubits": CFG.n_qubits_depth,
            "shots": CFG.shots_depth,
            "noise_type": depth_noise_type,
            "error_rate": float(depth_error_rate),
        }
    )
    run.set_tags({"experiment": "depth_scaling", "noise": depth_noise_type})

    for rep in CFG.depth_repetitions:
        circ = create_repeated_ghz_circuit(CFG.n_qubits_depth, rep)
        counts = sample_counts(sampler, circ, CFG.shots_depth)
        fid = compute_ghz_fidelity_from_counts(
            counts, CFG.n_qubits_depth, shots=CFG.shots_depth
        )

        circuit_depth = (
            len(circ) - 1
        )  # exclude the final measurement moment for depth proxy
        depth_points.append(
            {
                "repetitions": int(rep),
                "circuit_depth_moments": int(circuit_depth),
                "fidelity_proxy": float(fid),
            }
        )

        run.log_metric(
            "fidelity_proxy",
            float(fid),
            step=int(rep),
        )
        run.log_metric(
            "circuit_depth_moments",
            int(circuit_depth),
            step=int(rep),
        )

        print(f"repetitions={rep:2d}  depth={circuit_depth:3d}  fidelity={fid:.4f}")

    depth_run_id = run.run_id

print(f"Depth sweep run_id: {depth_run_id}")

## 5) Distribution distance (TVD) using devqubit comparisons

As a concrete “error budget” style comparison, we compare the baseline run against one noisy run and inspect:
- total variation distance (TVD) between recorded distributions (when available)
- the formatted devqubit comparison report


In [ ]:
# Pick one run from the sweep to compare against the ideal baseline
depol_run_id = next(
    r["run_id"] for r in sweep_results if r["noise_type"] == "depolarizing"
)

comparison = diff(
    ideal_run_id,
    depol_run_id,
    registry=registry,
    store=store,
)

print("Comparison: ideal vs depolarizing")
print("=" * 60)
print(f"Ideal run_id      : {ideal_run_id}")
print(f"Depolarizing run_id: {depol_run_id}")

if getattr(comparison, "tvd", None) is not None:
    print(f"\nTotal Variation Distance (TVD): {comparison.tvd:.4f}")
else:
    print("\nTVD not available for this comparison object.")

print("\nDetailed comparison report")
print("-" * 60)
print(comparison.format())

## 6) Quick summary

A tiny recap of what we just observed (based on the tracked runs).


In [ ]:
# Summarize sweep results (sorted best → worst)
print("Noise sweep summary")
print("-" * 60)
for rec in sorted(sweep_results, key=lambda r: r["fidelity_proxy"], reverse=True):
    loss = 1.0 - rec["fidelity_proxy"]
    print(
        f"{rec['noise_type']:20s}  "
        f"rate={rec['error_rate']:>7.4f}  "
        f"fidelity={rec['fidelity_proxy']:.4f}  "
        f"loss={loss:.4f}"
    )

print("\nDepth scaling points")
print("-" * 60)
for p in sorted(depth_points, key=lambda x: x["circuit_depth_moments"]):
    print(
        f"repetitions={p['repetitions']:2d}  "
        f"depth={p['circuit_depth_moments']:3d}  "
        f"fidelity={p['fidelity_proxy']:.4f}"
    )

print("\nTracked groups in this workspace:")
for g in registry.list_groups():
    print(f"- {g['group_name']} (group_id={g['group_id']})")

## 7) Cleanup

Remove the local devqubit workspace directory created for this demo.


In [ ]:
# Cleanup (optional): delete the local workspace directory
shutil.rmtree(WORKSPACE)
print(f"Removed workspace: {WORKSPACE}")